# Step 0.1. Import necessary libraries 

In [1]:
# Standard python libraries
import logging
import os
import time
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ImportanceCutoffSelector, ModelBasedImportanceEstimator
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
from lightautoml.utils.profiler import Profiler
from lightautoml.automl.blend import WeightedBlender

# Step 0.2. Parameters 

In [2]:
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TARGET_NAME = 'TARGET' # Target column name

# Step 0.3. Fix torch number of threads and numpy seed 

In [3]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

# Step 0.4. Change profiling decorators settings 

By default, profiling decorators are turned off for speed and memory reduction. If you want to see profiling report after using LAMA, you need to turn on the decorators using command below: 

In [4]:
p = Profiler()
p.change_deco_settings({'enabled': True})

ALL_FUNCS len = 378


# Step 0.5. Example data load 

In [5]:
%%time

data = pd.read_csv('./example_data/test_data_files/sampled_app_train.csv')
data.head()

CPU times: user 60.6 ms, sys: 19.8 ms, total: 80.3 ms
Wall time: 79.7 ms


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,313802,0,Cash loans,M,N,Y,0,270000.0,327024.0,15372.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,319656,0,Cash loans,F,N,N,0,108000.0,675000.0,19737.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,207678,0,Revolving loans,F,Y,Y,2,112500.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
3,381593,0,Cash loans,F,N,N,1,67500.0,142200.0,9630.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
4,258153,0,Cash loans,F,Y,Y,0,337500.0,1483231.5,46570.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0


# Step 0.6. (Optional) Some user feature preparation 

Cell below shows some user feature preparations to create task more difficult (this block can be omitted if you don't want to change the initial data):

In [6]:
%%time

data['BIRTH_DATE'] = (np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))).astype(str)
data['EMP_DATE'] = (np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
                    ).astype(str)

data['constant'] = 1
data['allnan'] = np.nan

data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1, inplace=True)

[2020-10-17 19:27:03,353] (INFO): Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2020-10-17 19:27:03,354] (INFO): Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2020-10-17 19:27:03,354] (INFO): NumExpr defaulting to 8 threads.


CPU times: user 108 ms, sys: 5.33 ms, total: 113 ms
Wall time: 111 ms


# Step 0.7. (Optional) Data splitting for train-test 

Block below can be omitted if you are going to train model only or you have specific train and test files:

In [7]:
%%time

train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE, 
                                         stratify=data[TARGET_NAME], 
                                         random_state=RANDOM_STATE)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)
logging.info('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

[2020-10-17 19:27:03,428] (INFO): Data splitted. Parts sizes: train_data = (8000, 124), test_data = (2000, 124)


CPU times: user 13.9 ms, sys: 0 ns, total: 13.9 ms
Wall time: 12.8 ms


In [8]:
train_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BIRTH_DATE,EMP_DATE,constant,allnan
0,112261,0,Cash loans,F,N,N,1,90000.0,640080.0,31261.5,...,0.0,0.0,0.0,0.0,1.0,0.0,1985-06-28,2012-06-21,1,NaN
1,115058,0,Cash loans,F,N,Y,0,180000.0,239850.0,23850.0,...,0.0,0.0,0.0,0.0,0.0,3.0,1953-12-27,2018-01-01,1,NaN
2,326623,0,Cash loans,F,N,Y,0,112500.0,337500.0,31086.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1975-06-21,2016-06-17,1,NaN
3,191976,0,Cash loans,M,Y,Y,1,67500.0,135000.0,9018.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1988-04-27,2009-06-05,1,NaN
4,281519,0,Revolving loans,F,N,Y,0,67500.0,202500.0,10125.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1975-06-13,1997-01-22,1,NaN


# ========= AutoML creation =========

![AutoML pipeline for this task](imgs/tutorial_1_pipeline.png)


## Step 1. Create Task and PandasReader

In [9]:
%%time

task = Task('binary')
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE)

CPU times: user 4.31 ms, sys: 0 ns, total: 4.31 ms
Wall time: 3.73 ms


## Step 2. Create feature selector (if necessary) 

In [10]:
%%time

model0 = BoostLGBM(
    default_params={'learning_rate': 0.05, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS}
)
pipe0 = LGBSimpleFeatures()
mbie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(pipe0, model0, mbie, cutoff=0)

CPU times: user 1.18 ms, sys: 0 ns, total: 1.18 ms
Wall time: 1.1 ms


/mnt/alex/lightautoml_17-10-2020_tutorials/lightautoml/utils/timer.py:208: UserWarning:

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer



## Step 3.1. Create 1st level ML pipeline for AutoML 

Our first level ML pipeline:
- Simple features for gradient boosting built on selected features (using step 2) 
- 2 different models:
    * LightGBM with params tuning (using OptunaTuner)
    * LightGBM with heuristic params


In [11]:
%%time 

pipe = LGBSimpleFeatures()

params_tuner1 = OptunaTuner(n_trials=20, timeout=30) # stop after 20 iterations or after 30 seconds 
model1 = BoostLGBM(
    default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS}
)
model2 = BoostLGBM(
    default_params={'learning_rate': 0.025, 'num_leaves': 64, 'seed': 2, 'num_threads': N_THREADS}
)

pipeline_lvl1 = MLPipeline([
    (model1, params_tuner1),
    model2
], pre_selection=selector, features_pipeline=pipe, post_selection=None)

CPU times: user 648 µs, sys: 0 ns, total: 648 µs
Wall time: 650 µs


## Step 3.2. Create 2nd level ML pipeline for AutoML 

Our second level ML pipeline:
- Using simple features as well, but now it will be Out-Of-Fold (OOF) predictions of algos from 1st level
- Only one LGBM model without params tuning
- Without feature selection on this stage because we want to use all OOFs here

In [12]:
%%time

pipe1 = LGBSimpleFeatures()

model = BoostLGBM(
    default_params={'learning_rate': 0.05, 'num_leaves': 64, 'max_bin': 1024, 'seed': 3, 'num_threads': N_THREADS},
    freeze_defaults=True
)

pipeline_lvl2 = MLPipeline([model], pre_selection=None, features_pipeline=pipe1, post_selection=None)

CPU times: user 171 µs, sys: 296 µs, total: 467 µs
Wall time: 468 µs


## Step 4. Create AutoML pipeline 

AutoML pipeline consist of:
- Reader for data preparation
- First level ML pipeline (as built in step 3.1)
- Second level ML pipeline (as built in step 3.2)
- `Skip_conn = False` equals here "not to use initial features on the second level pipeline"

In [13]:
%%time 

automl = AutoML(reader, [
    [pipeline_lvl1],
    [pipeline_lvl2],
], skip_conn=False)

CPU times: user 171 µs, sys: 296 µs, total: 467 µs
Wall time: 470 µs


## Step 5. Train AutoML on loaded data 

In cell below we train AutoML with target column `TARGET` to receive fitted model and OOF predictions:

In [14]:
%%time 

oof_pred = automl.fit_predict(train_data, roles={'target': TARGET_NAME})
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

/mnt/alex/lightautoml_17-10-2020_tutorials/lightautoml/reader/guess_roles.py:351: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Feats was rejected during automatic roles guess: []
Train process start. Time left 9999999996.342949 secs
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.716183
Early stopping, best iteration is:
[16]	valid's auc: 0.720694
Time history [0.5748012065887451]. Time left 8999999995.444473


/mnt/alex/lightautoml_17-10-2020_tutorials/lightautoml/utils/timer.py:208: UserWarning:

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer

[I 2020-10-17 19:27:08,359] A new study created in memory with name: no-name-4b02960a-af27-487d-89d3-b8a6a0e5898e


Optuna may run 6299999996.61487 secs
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.718332
[200]	valid's auc: 0.716862


[I 2020-10-17 19:27:09,969] Trial 0 finished with value: 0.7226454127042673 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 108}. Best is trial 0 with value: 0.7226454127042673.


...

[100]	valid's auc: 0.746925
Early stopping, best iteration is:
[49]	valid's auc: 0.751896


[I 2020-10-17 19:27:30,672] Trial 19 finished with value: 0.7518963388232275 and parameters: {'feature_fraction': 0.853875799774876, 'num_leaves': 33}. Best is trial 16 with value: 0.761539736249873.


Time history [0.6294820308685303]. Time left 8999999999.363365
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.755654
Early stopping, best iteration is:
[88]	valid's auc: 0.76154
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.726377
Early stopping, best iteration is:
[91]	valid's auc: 0.728547
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.72082
Early stopping, best iteration is:
[27]	valid's auc: 0.728147
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.719461
[200]	valid's auc: 0.721563
[300]	valid's auc: 0.721441
Early stopping, best iteration is:
[246]	valid's auc: 0.726695
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.751751
Early stopping, best iteration is:
[88]	valid's auc: 0.756146
Time history [0.48363518714904785, 0.49312496185302734, 0.3303530216217041, 0.877387523651123, 0.48085737228393555]. Time 

[2020-10-17 19:27:40,755] (INFO): oof_pred:
array([[0.07475057],
       [0.0326896 ],
       [0.0394009 ],
       ...,
       [0.02765582],
       [0.10423973],
       [0.17042336]], dtype=float32)
Shape = (8000, 1)


[100]	valid's auc: 0.683018
Early stopping, best iteration is:
[13]	valid's auc: 0.725899
Time history [0.3892176151275635, 0.42418336868286133, 0.32747340202331543, 0.4074714183807373, 0.3507721424102783]. Time left 8999999962.785194
Time left 9999999962.822792
CPU times: user 4min 20s, sys: 2.18 s, total: 4min 23s
Wall time: 37.2 s


## Step 6. Analyze fitted model  

Below we analyze feature importances of different algos:

In [15]:
logging.info('Feature importances of selector:\n{}'
              .format(selector.get_features_score()))
logging.info('=' * 70)

logging.info('Feature importances of top level algorithm:\n{}'
              .format(automl.levels[-1][0].ml_algos[0].get_features_score()))
logging.info('=' * 70)

logging.info('Feature importances of lowest level algorithm - model 0:\n{}'
              .format(automl.levels[0][0].ml_algos[0].get_features_score()))
logging.info('=' * 70)

logging.info('Feature importances of lowest level algorithm - model 1:\n{}'
              .format(automl.levels[0][0].ml_algos[1].get_features_score()))
logging.info('=' * 70)

[2020-10-17 19:27:40,764] (INFO): Feature importances of selector:
EXT_SOURCE_3              1029.681686
EXT_SOURCE_2               894.265428
BIRTH_DATE                 537.081401
EXT_SOURCE_1               424.764621
DAYS_LAST_PHONE_CHANGE     262.583100
                             ...     
FLAG_DOCUMENT_16             0.000000
FLAG_DOCUMENT_14             0.000000
FLAG_DOCUMENT_13             0.000000
FLAG_DOCUMENT_11             0.000000
FLAG_PHONE                   0.000000
Length: 110, dtype: float64
[2020-10-17 19:27:40,765] (INFO): ======================================================================
[2020-10-17 19:27:40,767] (INFO): Feature importances of top level algorithm:
Lvl_0_Pipe_0_Mod_0_LightGBM_prediction_0    2861.708537
Lvl_0_Pipe_0_Mod_1_LightGBM_prediction_0    2043.129412
dtype: float64
[2020-10-17 19:27:40,768] (INFO): ======================================================================
[2020-10-17 19:27:40,771] (INFO): Feature importances of lowest level al

## Step 7. Predict to test data and check scores

In [16]:
%%time

test_pred = automl.predict(test_data)
logging.info('Prediction for test data:\n{}\nShape = {}'
              .format(test_pred, test_pred.shape))

logging.info('Check scores...')
logging.info('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values, oof_pred.data[:, 0])))
logging.info('TEST score: {}'.format(roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])))

[2020-10-17 19:27:40,912] (INFO): Prediction for test data:
array([[0.04925682],
       [0.05368711],
       [0.0534688 ],
       ...,
       [0.04521164],
       [0.04988748],
       [0.16970594]], dtype=float32)
Shape = (2000, 1)
[2020-10-17 19:27:40,913] (INFO): Check scores...
[2020-10-17 19:27:40,916] (INFO): OOF score: 0.706138322789459
[2020-10-17 19:27:40,919] (INFO): TEST score: 0.7235682744565217


CPU times: user 476 ms, sys: 3.26 ms, total: 480 ms
Wall time: 133 ms


## Step 8. Profiling AutoML 

To build report here, we must turn on decorators on step 0.4. Report is interactive and you can go as deep into functions call stack as you want:

In [ ]:
%%time
p.profile('my_report_profile.html')
assert os.path.exists('my_report_profile.html'), 'Profile report failed to build'

# Appendix. Profiling report screenshots 

After loading HTML with profiling report, you can see fully folded report (please wait for green LOAD OK text for full load finish). If you click on triangle on the left, it unfolds and look like this:  

<img src="imgs/tutorial_1_initial_report.png" alt="Initial profiling report" style="width: 500px;"/>

If we go even deeper we will receive situation like this:

<img src="imgs/tutorial_1_unfolded_report.png" alt="Profiling report after several unfoldings on different levels" style="width: 500px;"/>
